In [1]:
import findspark
findspark.add_packages("graphframes:graphframes:0.8.0-spark3.0-s_2.12")
findspark.init()
import pyspark
import os
os.environ['PYSPARK_SUBMIT_ARGS']="--master local[2] pyspark-shell"
sc = pyspark.SparkContext(appName = "MyAPP")
from pyspark.sql.functions import *
from pyspark.sql.session import SparkSession
spark = SparkSession(sc)
sc.addPyFile('/home/msbd5003/graphframes-0.8.0-spark3.0-s_2.12.jar')

/home/msbd5003/spark/python/pyspark/context.py:220: DeprecationWarning: Support for Python 2 and Python 3 prior to version 3.6 is deprecated as of Spark 3.0. See also the plan for dropping Python 2 support at https://spark.apache.org/news/plan-for-dropping-python-2-support.html.
  DeprecationWarning)


In [61]:
import pandas as pd
import numpy as np
from datetime import date
from pyspark.sql.types import FloatType
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType

from pyspark.sql.functions import *

In [62]:
coupon3 = spark.read.csv('data/coupon3_feature.csv',header=True, inferSchema=True)
merchant3 = spark.read.csv('data/merchant3_feature.csv',header=True, inferSchema=True)
user3 = spark.read.csv('data/user3_feature.csv',header=True, inferSchema=True)
user_merchant3 = spark.read.csv('data/user_merchant3.csv',header=True, inferSchema=True)
other_feature3 = spark.read.csv('data/other_feature3.csv',header=True, inferSchema=True)

In [63]:
coupon1 = spark.read.csv('data/coupon1_feature.csv',header=True, inferSchema=True)
merchant1 = spark.read.csv('data/merchant1_feature.csv',header=True, inferSchema=True)
user1 = spark.read.csv('data/user1_feature.csv',header=True, inferSchema=True)
user_merchant1 = spark.read.csv('data/user_merchant1.csv',header=True, inferSchema=True)
other_feature1 = spark.read.csv('data/other_feature1.csv',header=True, inferSchema=True)

In [64]:
coupon2 = spark.read.csv('data/coupon2_feature.csv',header=True, inferSchema=True)
merchant2 = spark.read.csv('data/merchant2_feature.csv',header=True, inferSchema=True)
user2 = spark.read.csv('data/user2_feature.csv',header=True, inferSchema=True)
user_merchant2 = spark.read.csv('data/user_merchant2.csv',header=True, inferSchema=True)
other_feature2 = spark.read.csv('data/other_feature2.csv',header=True, inferSchema=True)

In [65]:
dataset3 = coupon3.join(merchant3,'Merchant_id')
dataset3 = dataset3.join(user3,'User_id')
#print("2")
#print(dataset3.columns)
dataset3 = dataset3.join(user_merchant3,['User_id','Merchant_id'])
#print("3")
#print(dataset3.columns)
dataset3 = dataset3.join(other_feature3,['User_id','Coupon_id','Date_received','Merchant_id'])
#print("4")
#print(dataset3.columns)
print(dataset3.count())
# only drop the row that is completely the same
dataset3 = dataset3.dropDuplicates()
print(dataset3.count())

1930723
7500


In [66]:
null_re = udf(lambda x: 0 if(x == 'null') else x)
weekend = udf(lambda x: 1 if(x in (6,7)) else 0)

In [67]:
dataset3 = dataset3.withColumn('User_merchant_buy_total', null_re(col('User_merchant_buy_total')))
dataset3 = dataset3.withColumn('User_merchant_any', null_re(col('User_merchant_any')))
dataset3 = dataset3.withColumn('User_merchant_received', null_re(col('User_merchant_received')))
dataset3 = dataset3.withColumn('Is_weekend', weekend(col('Day_of_week')))



Next session is to get weekdays dummies and concatenate dummies and original DF
hard to concat two spark DF, if find solution, 
then no need to tranfer to Pandas DF.

In [68]:
dataset3_pd = dataset3.toPandas()
weekday_dummies = pd.get_dummies(dataset3_pd.Day_of_week)
weekday_dummies.columns = ['Weekday'+str(i+1) for i in range(weekday_dummies.shape[1])]
dataset3 = spark.createDataFrame(pd.concat([dataset3_pd,weekday_dummies],axis=1))
drop_list = ['Merchant_id', 'Day_of_week','Coupon_count']
dataset3 = dataset3.drop(*drop_list)
dataset3 = dataset3.replace('null',str(np.nan))
print(dataset3.columns)


['User_id', 'Coupon_id', 'Date_received', 'Discount_rate', 'Distance', 'Day_of_mouth', 'Days_distance', 'Total_sales', 'Sales_use_coupon', 'Total_coupon', 'Merchant_min_distance', 'Merchant_max_distance', 'Merchant_mean_distance', 'Merchant_median_distance', 'Merchant_coupon_transfer_rate', 'Coupon_rate', 'count_merchant', 'user_min_distance', 'user_max_distance', 'user_mean_distance', 'user_median_distance', 'buy_use_coupon', 'buy_total', 'coupon_received', 'avg_user_date_datereceived_gap', 'min_user_date_datereceived_gap', 'max_user_date_datereceived_gap', 'buy_use_coupon_rate', 'User_merchant_buy_total', 'User_merchant_received', 'User_merchant_buy_use_coupon', 'User_merchant_any', 'User_merchant_buy_common', 'User_merchant_coupon_transfer_rate', 'User_merchant_coupon_buy_rate', 'User_merchant_coupon_rate', 'User_merchant_coupon_common_rate', 'Month_Receive_same_coupon_count', 'Month_Receive_all_coupon_count', 'Month_same_coupon_lastone', 'Month_same_coupon_firstone', 'Day_Receive_a

In [69]:
dataset3.repartition(1).write.csv('dataset3.csv', header = True)

In [70]:
dataset2 = coupon2.join(merchant2,'Merchant_id')
dataset2 = dataset2.join(user2,'User_id')

dataset2 = dataset2.join(user_merchant2,['User_id','Merchant_id'])

dataset2 = dataset2.join(other_feature2,['User_id','Coupon_id','Date_received','Merchant_id'])

print(dataset2.count())
# only drop the row that is completely the same
dataset2 = dataset2.dropDuplicates()
print(dataset2.count())
print(dataset2.columns)

3352809
5594
['User_id', 'Coupon_id', 'Date_received', 'Merchant_id', 'Discount_rate', 'Distance', 'Date', 'Day_of_week', 'Day_of_mouth', 'Days_distance', 'Coupon_count', 'Total_sales', 'Sales_use_coupon', 'Total_coupon', 'Merchant_min_distance', 'Merchant_max_distance', 'Merchant_mean_distance', 'Merchant_median_distance', 'Merchant_coupon_transfer_rate', 'Coupon_rate', 'count_merchant', 'user_min_distance', 'user_max_distance', 'user_mean_distance', 'user_median_distance', 'buy_use_coupon', 'buy_total', 'coupon_received', 'avg_user_date_datereceived_gap', 'min_user_date_datereceived_gap', 'max_user_date_datereceived_gap', 'buy_use_coupon_rate', 'user_coupon_transfer_rate', 'User_merchant_buy_total', 'User_merchant_received', 'User_merchant_buy_use_coupon', 'User_merchant_any', 'User_merchant_buy_common', 'User_merchant_coupon_transfer_rate', 'User_merchant_coupon_buy_rate', 'User_merchant_coupon_rate', 'User_merchant_coupon_common_rate', 'Label_user_merchant_coupon_count', 'Label_mer

In [71]:
dataset2 = dataset2.withColumn('User_merchant_buy_total', null_re(col('User_merchant_buy_total')))
dataset2 = dataset2.withColumn('User_merchant_any', null_re(col('User_merchant_any')))
dataset2 = dataset2.withColumn('User_merchant_received', null_re(col('User_merchant_received')))
dataset2 = dataset2.withColumn('Is_weekend', weekend(col('Day_of_week')))



In [72]:
split_col = udf(lambda x: 'null' if x is None else x.split(':'))
get_label = udf(lambda x: 0 if x[0]=='null' else (\
            1 if (date(int(float(x[0][0:4])), int(float(x[0][4:6])), int(float(x[0][6:8])))-date(int(float(x[1][0:4])),int(float(x[1][4:6])),int(float(x[1][6:8])))).days<=15\
              else -1))
add_date = udf(lambda x,y: 'null' if ((x is None) or (y is None)) else str(x)+':'+str(y))

In [73]:
dataset2_pd = dataset2.toPandas()
weekday_dummies = pd.get_dummies(dataset2_pd.Day_of_week)
weekday_dummies.columns = ['Weekday'+str(i+1) for i in range(weekday_dummies.shape[1])]
dataset2_pd
dataset2 = spark.createDataFrame(pd.concat([dataset2_pd,weekday_dummies],axis=1))
dataset2.select('Date').show(10)
dataset2.select('Date_received').show(10)

###
dataset2 = dataset2.withColumn('Label',add_date(col('Date'),col('Date_received')))
dataset2.select('Label').show(10)
#dataset2.select('Label'),printSchema()
#dataset2 = dataset2.withColumn('Label',col('Label')+col('Date_received').cast(StringType()))
#dataset2.select('Label').show(10)
#dataset2.select('Label'),printSchema()
#print(dataset2.Label.)
#dataset2 = dataset2.withColumn('Label', col('Label').cast(StringType()))
#dataset2.select('Label').show(10)
dataset2 = dataset2.withColumn('Label',split_col(col('Label'))) 
dataset2.select('Label').show(10)
dataset2 = dataset2.withColumn('Label',get_label(col('Label')))
#dataset2.printSchema()
dataset2.select('Label').show(10)

+--------+
|    Date|
+--------+
|20160522|
|20160610|
|    null|
|    null|
|    null|
|    null|
|20160530|
|20160620|
|20160519|
|20160520|
+--------+
only showing top 10 rows

+-------------+
|Date_received|
+-------------+
|     20160521|
|     20160521|
|     20160516|
|     20160610|
|     20160525|
|     20160530|
|     20160516|
|     20160530|
|     20160519|
|     20160516|
+-------------+
only showing top 10 rows

+-----------------+
|            Label|
+-----------------+
|20160522:20160521|
|20160610:20160521|
|    null:20160516|
|    null:20160610|
|    null:20160525|
|    null:20160530|
|20160530:20160516|
|20160620:20160530|
|20160519:20160519|
|20160520:20160516|
+-----------------+
only showing top 10 rows

+--------------------+
|               Label|
+--------------------+
|[20160522, 20160521]|
|[20160610, 20160521]|
|    [null, 20160516]|
|    [null, 20160610]|
|    [null, 20160525]|
|    [null, 20160530]|
|[20160530, 20160516]|
|[20160620, 20160530]|
|[20160519,

In [74]:
drop_list = ['Merchant_id', 'Day_of_week','Coupon_count','Date','Date_recived','Coupon_id','Coupon_count']
dataset2 = dataset2.drop(*drop_list)                               
dataset2 = dataset2.replace('null',str(np.nan))
print(dataset2.columns)

['User_id', 'Date_received', 'Discount_rate', 'Distance', 'Day_of_mouth', 'Days_distance', 'Total_sales', 'Sales_use_coupon', 'Total_coupon', 'Merchant_min_distance', 'Merchant_max_distance', 'Merchant_mean_distance', 'Merchant_median_distance', 'Merchant_coupon_transfer_rate', 'Coupon_rate', 'count_merchant', 'user_min_distance', 'user_max_distance', 'user_mean_distance', 'user_median_distance', 'buy_use_coupon', 'buy_total', 'coupon_received', 'avg_user_date_datereceived_gap', 'min_user_date_datereceived_gap', 'max_user_date_datereceived_gap', 'buy_use_coupon_rate', 'user_coupon_transfer_rate', 'User_merchant_buy_total', 'User_merchant_received', 'User_merchant_buy_use_coupon', 'User_merchant_any', 'User_merchant_buy_common', 'User_merchant_coupon_transfer_rate', 'User_merchant_coupon_buy_rate', 'User_merchant_coupon_rate', 'User_merchant_coupon_common_rate', 'Label_user_merchant_coupon_count', 'Label_merchant_coupon_count', 'Coupon_count_later', 'Month_Receive_same_coupon_count', 'M

In [75]:
# could not convert string to float
dataset2.repartition(1).write.csv('dataset2.csv', header = True)

In [76]:
dataset1 = coupon1.join(merchant1,'Merchant_id')
dataset1 = dataset1.join(user1,'User_id')

dataset1 = dataset1.join(user_merchant1,['User_id','Merchant_id'])

dataset1 = dataset1.join(other_feature1,['User_id','Coupon_id','Date_received','Merchant_id'])

print(dataset1.count())
# only drop the row that is completely the same
dataset1 = dataset1.dropDuplicates()
print(dataset1.count())

3118743
1994


In [77]:
dataset1 = dataset1.withColumn('User_merchant_buy_total', null_re(col('User_merchant_buy_total')))
dataset1 = dataset1.withColumn('User_merchant_any', null_re(col('User_merchant_any')))
dataset1 = dataset1.withColumn('User_merchant_received', null_re(col('User_merchant_received')))
dataset1 = dataset1.withColumn('Is_weekend', weekend(col('Day_of_week')))



In [78]:
dataset1_pd = dataset1.toPandas()
weekday_dummies = pd.get_dummies(dataset1_pd.Day_of_week)
weekday_dummies.columns = ['Weekday'+str(i+1) for i in range(weekday_dummies.shape[1])]
dataset1_pd
dataset1 = spark.createDataFrame(pd.concat([dataset1_pd,weekday_dummies],axis=1))
dataset1.select('Date').show(10)
dataset1.select('Date_received').show(10)

###
dataset1 = dataset1.withColumn('Label',add_date(col('Date'),col('Date_received')))
dataset1.select('Label').show(10)
#dataset2.select('Label'),printSchema()
#dataset2 = dataset2.withColumn('Label',col('Label')+col('Date_received').cast(StringType()))
#dataset2.select('Label').show(10)
#dataset2.select('Label'),printSchema()
#print(dataset2.Label.)
#dataset2 = dataset2.withColumn('Label', col('Label').cast(StringType()))
#dataset2.select('Label').show(10)
dataset1 = dataset1.withColumn('Label',split_col(col('Label'))) 
dataset1.select('Label').show(10)
dataset1 = dataset1.withColumn('Label',get_label(col('Label')))
#dataset2.printSchema()
dataset1.select('Label').show(10)

+--------+
|    Date|
+--------+
|    null|
|20160607|
|20160420|
|20160506|
|    null|
|    null|
|    null|
|    null|
|    null|
|    null|
+--------+
only showing top 10 rows

+-------------+
|Date_received|
+-------------+
|     20160504|
|     20160514|
|     20160420|
|     20160429|
|     20160414|
|     20160422|
|     20160425|
|     20160425|
|     20160425|
|     20160429|
+-------------+
only showing top 10 rows

+-----------------+
|            Label|
+-----------------+
|    null:20160504|
|20160607:20160514|
|20160420:20160420|
|20160506:20160429|
|    null:20160414|
|    null:20160422|
|    null:20160425|
|    null:20160425|
|    null:20160425|
|    null:20160429|
+-----------------+
only showing top 10 rows

+--------------------+
|               Label|
+--------------------+
|    [null, 20160504]|
|[20160607, 20160514]|
|[20160420, 20160420]|
|[20160506, 20160429]|
|    [null, 20160414]|
|    [null, 20160422]|
|    [null, 20160425]|
|    [null, 20160425]|
|    [null,

In [79]:
drop_list = ['Merchant_id', 'Day_of_week','Coupon_count','Date','Date_recived','Coupon_id','Coupon_count']
dataset1 = dataset1.drop(*drop_list)                               
dataset1 = dataset1.replace('null',str(np.nan))
#print(dataset1.columns)


In [80]:
dataset1.repartition(1).write.csv('dataset1.csv', header = True)